### Newton–Cotes quadrature rules

In [22]:
from IPython.display import display, Math

def check_args(xs, ys, odd = False):
    if len(xs) != len(ys):
        raise Exception("invalid args len")
    if odd and (len(xs) % 2 == 0):
        raise Exception("This rule requires odd number of points")

def integrate_by_trapezoids(xs, ys):
    check_args(xs, ys)
    n = len(xs)
    I = 0
    for i in range(n - 1):
        I += (xs[i + 1] - xs[i]) * (ys[i + 1] + ys[i]) / 2
    return I

def integrate_by_Simpson(xs, ys):
    check_args(xs, ys, odd=True)
    n = len(xs)
    I = 0
    for i in range(0, n - 2, 2):
        I += (xs[i + 2] - xs[i]) * (ys[i + 2] + 4*ys[i + 1] + ys[i]) / 6
    return I

def apply_Runge_rule(xs, ys, method, order):
    check_args(xs, ys, odd=True)
    I = method(xs, ys)
    I2 = method(xs[::2], ys[::2])
    return I + (I - I2) / (2**order - 1)

xs = [0, 0.25, 0.5, 0.75, 1.0, 1.25, 1.5, 1.75, 2.0]
fs = [1.0000000, 0.989616, 0.958851, 0.908852, 0.841471, 0.759188, 0.664997, 0.562278, 0.454649]

print("Метод трапеций:")
display(Math(f"I_{{tr}} = {integrate_by_trapezoids(xs, fs)}"))
print("Метод Симпсона:")
display(Math(f"I_{{Sp}} = {integrate_by_Simpson(xs, fs)}"))
print("Правило Рунге для метода трапеций:")
display(Math(f"I_{{Rg}} = {apply_Runge_rule(xs, fs, integrate_by_trapezoids, 2)}"))
print("Отсюда видно, что правило Рунге делает из метода трапеций метод Симпсона.")

Метод трапеций:


<IPython.core.display.Math object>

Метод Симпсона:


<IPython.core.display.Math object>

Правило Рунге для метода трапеций:


<IPython.core.display.Math object>

Отсюда видно, что правило Рунге делает из метода трапеций метод Симпсона.
